In [1]:
# config.py
API_KEY = "04RRGEKMK1FDDJYP"

In [6]:
# data_fetch_alpha_vantage.py
import requests
import pandas as pd

def fetch_stock_data(symbol, outputsize='full'):
    url = (
        f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}"
        f"&outputsize={outputsize}&apikey={API_KEY}&datatype=csv"
    )
    r = requests.get(url)
    filename = f"data_{symbol}.csv"
    with open(filename, "w") as f:
        f.write(r.text)
    print(f"Saved {symbol} data to {filename}")
    return filename

# Example usage:
if __name__ == "__main__":
    fetch_stock_data("AAPL")


Saved AAPL data to data_AAPL.csv


In [7]:
# var_cvar_delta_normal.py
import pandas as pd
import numpy as np
from scipy.stats import norm

confidence_level = 0.95
z = norm.ppf(1 - confidence_level)

def calculate_var_cvar(file, investment):
    df = pd.read_csv(file)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df.sort_index(inplace=True)
    
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df.dropna(inplace=True)
    
    mu = df['log_return'].mean()
    sigma = df['log_return'].std()
    
    VaR = -investment * (mu + z * sigma)
    CVaR = -investment * (mu + sigma * norm.pdf(z) / (1 - confidence_level))
    
    print(f"VaR (95%): {VaR:.2f}")
    print(f"CVaR (95%): {CVaR:.2f}")
    return VaR, CVaR

# Example
if __name__ == "__main__":
    calculate_var_cvar("data_AAPL.csv", investment=100000)


VaR (95%): 6643.10
CVaR (95%): -8363.70


In [8]:
# var_cvar_historical.py
import pandas as pd
import numpy as np

def historical_var_cvar(file, investment, confidence_level=0.95):
    df = pd.read_csv(file)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df.sort_index(inplace=True)
    
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df.dropna(inplace=True)

    sorted_returns = df['log_return'].sort_values()
    var_index = int((1 - confidence_level) * len(sorted_returns))
    VaR = -investment * sorted_returns.iloc[var_index]
    CVaR = -investment * sorted_returns.iloc[:var_index].mean()

    print(f"Historical VaR (95%): {VaR:.2f}")
    print(f"Historical CVaR (95%): {CVaR:.2f}")
    return VaR, CVaR

# Example
if __name__ == "__main__":
    historical_var_cvar("data_AAPL.csv", investment=100000)


Historical VaR (95%): 3724.64
Historical CVaR (95%): 7082.84


In [9]:
# var_cvar_monte_carlo.py
import pandas as pd
import numpy as np
from scipy.stats import norm

def monte_carlo_var_cvar(file, investment, confidence_level=0.95, num_simulations=10000):
    df = pd.read_csv(file)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df.sort_index(inplace=True)

    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df.dropna(inplace=True)

    mu = df['log_return'].mean()
    sigma = df['log_return'].std()

    simulated_returns = np.random.normal(mu, sigma, num_simulations)
    simulated_losses = -investment * simulated_returns

    VaR = np.percentile(simulated_losses, (1 - confidence_level) * 100)
    CVaR = simulated_losses[simulated_losses >= VaR].mean()

    print(f"Monte Carlo VaR (95%): {VaR:.2f}")
    print(f"Monte Carlo CVaR (95%): {CVaR:.2f}")
    return VaR, CVaR

# Example
if __name__ == "__main__":
    monte_carlo_var_cvar("data_AAPL.csv", investment=100000)


Monte Carlo VaR (95%): -6658.38
Monte Carlo CVaR (95%): 401.97
